In [8]:
# If needed:
# !pip install pandas numpy matplotlib openpyxl statsmodels

import re, warnings
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter
from statsmodels.tsa.arima.model import ARIMA
warnings.filterwarnings("ignore")

# ---------- YOUR FILE & OUTPUT ROOT ----------
FILE_PATH = r"D:\arima project\Task3\Task 3\cdc (IHD AND VFF) finals.xlsx"
BASE_RESULTS_DIR = Path(r"D:\arima project\Task3\Task 3\results")
BASE_RESULTS_DIR.mkdir(parents=True, exist_ok=True)

# Forecast horizon
END_YEAR = 2043

# Optional: if a sheet needs a specific AAMR column, put it here (by sheet name, exact)
MANUAL_AAMR_BY_SHEET = {
    # "overall final": "Age Adjusted Rate",   # example if auto-pick ever fails
}

# For very crowded sheets (e.g., states), show only top N groups in the combined plot (None = all)
TOP_N_IN_COMBINED = {
    "state final": 12,   # keep combined figure readable
}


In [9]:
plt.rcParams.update({
    "figure.figsize": (14, 8),
    "figure.dpi": 160,
    "axes.grid": True,
    "grid.alpha": 0.25,
    "font.size": 16,
    "font.weight": "bold",
    "axes.labelsize": 18,
    "axes.labelweight": "bold",
    "axes.titlesize": 24,
    "axes.titleweight": "bold",
    "legend.fontsize": 14,
    "legend.frameon": True,
    "legend.title_fontsize": 15,
    "lines.linewidth": 2.8,
    "lines.markersize": 7.0,
})

def style_axes(ax):
    ax.tick_params(axis="both", labelsize=16, width=2.2, length=7)
    for s in ax.spines.values():
        s.set_linewidth(2.2)


In [10]:
def safe_name(s: str) -> str:
    return re.sub(r"[^a-zA-Z0-9_-]+", "_", str(s)).strip("_")

def _flatten_columns(df: pd.DataFrame) -> pd.DataFrame:
    """Flatten multirow headers, normalize whitespace, deduplicate names."""
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = [
            " ".join([str(x) for x in tup if (str(x) != "nan" and str(x).strip() != "")]).strip()
            for tup in df.columns.to_list()
        ]
    df.columns = [re.sub(r"\s+", " ", str(c)).strip() for c in df.columns]
    seen, newcols = {}, []
    for c in df.columns:
        if c in seen:
            seen[c] += 1
            newcols.append(f"{c}.{seen[c]}")
        else:
            seen[c] = 0
            newcols.append(c)
    df.columns = newcols
    return df

def _pick_col(df, patterns, required=True, exclude_regex=None):
    low = {c: str(c).strip().lower() for c in df.columns}
    for pat in patterns:
        rx = re.compile(pat)
        for c, lc in low.items():
            if rx.search(lc):
                if exclude_regex and re.search(exclude_regex, lc):
                    continue
                return c
    if required:
        raise KeyError(f"Could not find any of: {patterns}")
    return None

def find_year(df):  # prefer 'Year'
    return _pick_col(df, [r"\byear\b", r"^yr$", r"\bcalendar\s*year\b"])

def find_group(df):
    """Likely grouping column (Sex/Age group/Race/Region/Urbanization/State/Variable)."""
    exclude = r"age\s*adjust|aamr|rate|ci|confidence|se|std|mean|median|total\s*deaths|deaths"
    patterns = [
        r"\bvariable\b",
        r"\bage\s*group(s)?\b|age\s*cat(egory|egories)?\b|age\s*grp\b|age\-group",
        r"\bgender\b|\bsex\b",
        r"\brace\b|ethnic|hispanic",
        r"\bregion\b|census",
        r"\burban(ization)?\b|rural\b",
        r"\bstate\b|^us state$|^state name$",
        r"\boverall\b|^total$"
    ]
    for pat in patterns:
        c = _pick_col(df, [pat], required=False, exclude_regex=exclude)
        if c:
            return c
    return None  # overall-only sheet

def find_aamr(df, sheet_name=None):
    # Manual override per sheet (if provided)
    if sheet_name and sheet_name in MANUAL_AAMR_BY_SHEET:
        col = MANUAL_AAMR_BY_SHEET[sheet_name]
        if col in df.columns:
            return col

    # 1) Precise names
    precise = [
        r"\bage[-\s]*adjust(ed)?\s*rate\b",
        r"\baamr\b",
        r"\bage[-\s]*standard(ized)?\s*rate\b"
    ]
    for p in precise:
        try:
            return _pick_col(df, [p])
        except:
            pass

    # 2) Generic 'Age Adjust*' that LOOKS like a rate (numeric, not %)
    bad_words = ("death", "%", "percent", "share")
    candidates = []
    for c in df.columns:
        lc = str(c).lower()
        if "age" in lc and "adjust" in lc and not any(b in lc for b in bad_words):
            # Prefer columns that are mostly numeric
            vals = pd.to_numeric(df[c], errors="coerce")
            nn = vals.notna().sum()
            if nn >= max(5, int(0.5 * len(df))):  # many numeric entries
                candidates.append((nn, c))
    if candidates:
        # pick the one with the MOST numeric values (usually the main rate)
        candidates.sort(reverse=True)
        return candidates[0][1]

    # 3) Last resort: any numeric column with 'rate' in name
    for c in df.columns:
        lc = str(c).lower()
        if "rate" in lc:
            vals = pd.to_numeric(df[c], errors="coerce")
            if vals.notna().sum() >= 5:
                return c

    raise KeyError("AAMR column not found. Set MANUAL_AAMR_BY_SHEET for this sheet.")


In [11]:
def load_observed_excel(path, sheet_name, overall_label="Overall"):
    xls = pd.ExcelFile(path)
    if sheet_name not in xls.sheet_names:
        raise ValueError(f"Sheet '{sheet_name}' not found. Options: {xls.sheet_names}")

    df = pd.read_excel(xls, sheet_name=sheet_name)
    df = _flatten_columns(df)

    ycol = find_year(df)
    tcol = find_aamr(df, sheet_name=sheet_name)
    gcol = find_group(df)  # may be None

    df[ycol] = pd.to_numeric(df[ycol], errors="coerce")
    df[tcol] = pd.to_numeric(df[tcol], errors="coerce")

    if gcol and gcol == tcol:
        gcol = None

    if gcol is None:
        tidy = (df[[ycol, tcol]]
                .dropna(subset=[ycol, tcol])
                .groupby(ycol, as_index=False)[tcol].mean()
                .rename(columns={ycol:"Year", tcol:"AAMR"}))
        tidy["Group"] = overall_label
        tidy = tidy[["Year","Group","AAMR"]]
    else:
        df[gcol] = df[gcol].astype(str).str.strip()
        tidy = (df[[ycol, gcol, tcol]]
                .dropna(subset=[ycol, gcol, tcol])
                .groupby([ycol, gcol], as_index=False)[tcol].mean()
                .rename(columns={ycol:"Year", gcol:"Group", tcol:"AAMR"}))

    tidy["Year"] = tidy["Year"].astype(int)
    tidy = tidy.sort_values(["Group","Year"]).reset_index(drop=True)

    print(f"\nSheet: {sheet_name}")
    print(f"Detected → Year: '{ycol}' | AAMR: '{tcol}' | Group: '{(gcol or overall_label)}'")
    print(tidy.head(6))
    return tidy


In [12]:
def _sanitize_series_for_arima(y: pd.Series):
    y = y.sort_index()
    full = pd.Index(range(int(y.index.min()), int(y.index.max())+1), name=y.index.name)
    y = y.reindex(full)
    if y.isna().any():
        y = y.interpolate(limit_direction="both")
    return y

def select_arima_order(y):
    best = None
    for d in [0,1,2]:
        for p in range(0,4):
            for q in range(0,4):
                if (p,d,q) == (0,0,0):
                    continue
                try:
                    trend = "n" if d>0 else "c"
                    res = ARIMA(y, order=(p,d,q), trend=trend,
                                enforce_stationarity=False, enforce_invertibility=False
                               ).fit(method_kwargs={"warn_convergence":False})
                    score = res.aic
                    if (best is None) or (score < best[0]):
                        best = (score, (p,d,q,trend), res)
                except Exception:
                    pass
    if best is None:
        res = ARIMA(y, order=(1,1,0), trend="n",
                    enforce_stationarity=False, enforce_invertibility=False
                   ).fit(method_kwargs={"warn_convergence":False})
        return (1,1,0,"n"), res
    return best[1], best[2]

def forecast_to(y, end_year, conf=0.95):
    last_year = int(y.index.max())
    steps = max(0, end_year - last_year)
    if steps == 0:
        raise ValueError("Observed series already extends to END_YEAR.")
    order, res = select_arima_order(y)
    fc = res.get_forecast(steps=steps)
    mean = fc.predicted_mean
    ci   = fc.conf_int(alpha=1-conf)
    lo, hi = ci.iloc[:,0], ci.iloc[:,1]
    yrs = list(range(last_year+1, end_year+1))
    out = pd.DataFrame({
        "Year": yrs,
        "Point.Forecast": mean.values,
        "Lo.95": lo.values,
        "Hi.95": hi.values,
        "Order": [f"{order[0]},{order[1]},{order[2]} ({order[3]})"]*steps
    })
    return order, out


In [13]:
def fit_all_groups_and_save(observed_df, end_year, out_dir: Path, title_prefix="AAMR"):
    out_dir.mkdir(parents=True, exist_ok=True)
    groups = sorted(observed_df["Group"].unique())
    all_rows = []
    last_obs_year = int(observed_df["Year"].max())

    for g in groups:
        sub = observed_df[observed_df["Group"]==g].copy()
        sub = sub.dropna(subset=["Year","AAMR"]).sort_values("Year")
        y = pd.Series(sub["AAMR"].values, index=sub["Year"].astype(int), name="AAMR")
        y = _sanitize_series_for_arima(y)

        order, fc = forecast_to(y, end_year=end_year, conf=0.95)

        # Save per-group CSV (2 decimals)
        csv_path = out_dir / f"{safe_name(g)}_forecast_to_{end_year}.csv"
        fc_out = fc.copy()
        for c in ["Point.Forecast","Lo.95","Hi.95"]:
            fc_out[c] = fc_out[c].round(2)
        fc_out.insert(0, "Series", g)
        fc_out.to_csv(csv_path, index=False)
        print(f"[{g}] order={order}  -> CSV:", csv_path)

        # Per-group plot with CI (no annotations at the end)
        fig, ax = plt.subplots()
        ax.plot(y.index, y.values, marker="o", label=f"{g} (obs)")
        ln, = ax.plot(fc["Year"], fc["Point.Forecast"], linestyle="--", marker="o", label=f"{g} (fc)")
        ax.fill_between(fc["Year"], fc["Lo.95"], fc["Hi.95"], alpha=0.14, color=ln.get_color(), label="95% PI (fc)")
        ax.axvline(x=last_obs_year+0.5, linestyle=":", linewidth=2.2)
        ax.set_title(f"{title_prefix}: {g} — observed (≤{last_obs_year}) & ARIMA forecast ({last_obs_year+1}–{end_year})")
        ax.set_xlabel("Year"); ax.set_ylabel("AAMR (per 100,000)")
        ax.yaxis.set_major_formatter(FormatStrFormatter("%.2f"))
        style_axes(ax)
        ax.legend(ncols=2)
        plt.tight_layout()
        png_path = out_dir / f"{safe_name(g)}_timeseries_to_{end_year}.png"
        plt.savefig(png_path, dpi=300, bbox_inches="tight"); plt.close()
        print("   Plot:", png_path)

        # keep for consolidated
        tmp = fc_out.copy(); tmp["Group"] = g
        all_rows.append(tmp)

    if all_rows:
        all_df = pd.concat(all_rows, ignore_index=True)
        all_df.to_csv(out_dir / f"ALL_GROUPS_forecasts_to_{end_year}.csv", index=False)
        print("Consolidated CSV:", out_dir / f"ALL_GROUPS_forecasts_to_{end_year}.csv")


In [14]:
def plot_combined(observed_df, out_dir: Path, end_year: int, title="AAMR", top_n=None):
    last_obs_year = int(observed_df["Year"].max())
    groups = sorted(observed_df["Group"].unique())

    # Optionally restrict to top N by last observed AAMR
    if top_n is not None and len(groups) > top_n:
        last_vals = (observed_df.sort_values("Year")
                     .groupby("Group")["AAMR"].last().sort_values(ascending=False))
        keep = set(last_vals.head(top_n).index.tolist())
        observed_df = observed_df[observed_df["Group"].isin(keep)]
        groups = sorted(keep)

    # Read saved forecasts for CI
    fc_map = {}
    for g in groups:
        p = out_dir / f"{safe_name(g)}_forecast_to_{end_year}.csv"
        if p.exists():
            fc_map[g] = pd.read_csv(p)

    fig, ax = plt.subplots()
    ci_legend_added = False

    for g in groups:
        sub = observed_df[observed_df["Group"]==g].copy().sort_values("Year")
        ax.plot(sub["Year"], sub["AAMR"], marker="o", label=f"{g} (obs)")
        if g in fc_map:
            fc = fc_map[g]
            ln, = ax.plot(fc["Year"], fc["Point.Forecast"], linestyle="--", marker="o", label=f"{g} (fc)")
            if not ci_legend_added:
                ax.fill_between(fc["Year"], fc["Lo.95"], fc["Hi.95"], alpha=0.12, color=ln.get_color(), label="95% PI (fc)")
                ci_legend_added = True
            else:
                ax.fill_between(fc["Year"], fc["Lo.95"], fc["Hi.95"], alpha=0.12, color=ln.get_color())

    ax.axvline(x=last_obs_year+0.5, linestyle=":", linewidth=2.2)
    ax.set_title(f"{title}: observed (≤{last_obs_year}) & ARIMA forecast ({last_obs_year+1}–{end_year})")
    ax.set_xlabel("Year"); ax.set_ylabel("AAMR (per 100,000)")
    ax.yaxis.set_major_formatter(FormatStrFormatter("%.2f"))
    style_axes(ax)
    ax.legend(ncols=2)
    ax.margins(x=0.02)
    plt.tight_layout()
    out = out_dir / f"COMBINED_timeseries_to_{end_year}.png"
    plt.savefig(out, dpi=300, bbox_inches="tight"); plt.close()
    print("Combined plot:", out)


In [15]:
# Map sheet name -> nice folder name & plot title
RUN_SHEETS = [
    ("overall final",       "overall",       "Overall"),
    ("Race final",          "race",          "Race/Ethnicity"),
    ("census final",        "region",        "Census Region"),
    ("state final",         "state",         "States"),
    ("urbanization final",  "urbanization",  "Urbanization"),
    ("age group final",     "age",           "Age groups"),
]

for sheet_name, folder, pretty_title in RUN_SHEETS:
    out_dir = BASE_RESULTS_DIR / folder
    out_dir.mkdir(parents=True, exist_ok=True)

    # 1) Load & tidy
    observed = load_observed_excel(FILE_PATH, sheet_name=sheet_name, overall_label="Overall")

    # 2) Fit & save per group (CSV + per-group plot)
    fit_all_groups_and_save(observed, END_YEAR, out_dir, title_prefix=pretty_title)

    # 3) Combined figure (optionally limit groups)
    topn = TOP_N_IN_COMBINED.get(sheet_name)
    plot_combined(observed, out_dir, END_YEAR, title=pretty_title, top_n=topn)

print("\nAll done. Files saved under:", BASE_RESULTS_DIR)



Sheet: overall final
Detected → Year: 'Year' | AAMR: 'Age Adjusted Rate' | Group: 'Overall'
   Year    Group      AAMR
0  1999  Overall  6.916667
1  2000  Overall  6.166667
2  2001  Overall  5.406667
3  2002  Overall  4.800000
4  2003  Overall  4.310000
5  2004  Overall  3.853333
[Overall] order=(1, 0, 0, 'c')  -> CSV: D:\arima project\Task3\Task 3\results\overall\Overall_forecast_to_2043.csv
   Plot: D:\arima project\Task3\Task 3\results\overall\Overall_timeseries_to_2043.png
Consolidated CSV: D:\arima project\Task3\Task 3\results\overall\ALL_GROUPS_forecasts_to_2043.csv
Combined plot: D:\arima project\Task3\Task 3\results\overall\COMBINED_timeseries_to_2043.png


ValueError: Sheet 'Race final' not found. Options: ['overall final', 'Race final ', 'census final', 'state final', 'urbanization final', 'age group final']

In [16]:
# ==== ARIMA FORECASTS & PLOTS, BY SHEET (to 2043) ====
# If needed first time:
# !pip install pandas numpy matplotlib openpyxl statsmodels

import os, re, warnings
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter
from statsmodels.tsa.arima.model import ARIMA
warnings.filterwarnings("ignore")

# ----------- USER SETTINGS -----------
FILE_PATH   = r"D:\arima project\Task3\Task 3\cdc (IHD AND VFF) finals.xlsx"
OUTPUT_BASE = Path(r"D:\arima project\Task3\Task 3\results")
END_YEAR    = 2043

# Sheet list → (sheet_name_as_you_see_it, folder_name, pretty_title)
# (sheet name matching ignores case and extra spaces)
RUN_SHEETS = [
    ("overall final",        "overall",        "Overall"),
    ("Race final",           "race",           "Race / Ethnicity"),
    ("census final",         "region",         "Census Regions"),
    ("state final",          "state",          "States"),
    ("urbanization final",   "urbanization",   "Urbanization"),
    ("age group final",      "age",            "Age Groups"),
]

# For very large categories (e.g., States), optionally limit combined chart
# to the top N groups by the latest observed AAMR (None = show all).
TOP_N_FOR_COMBINED = {
    "state": 12  # show top 12 states; change or remove as you like
}

# Plot style (bold & large)
plt.rcParams.update({
    "figure.figsize": (13, 7.6),
    "figure.dpi": 160,
    "axes.grid": True,
    "grid.alpha": 0.25,
    "font.size": 14,
    "font.weight": "bold",
    "axes.labelsize": 18,
    "axes.labelweight": "bold",
    "axes.titlesize": 22,
    "axes.titleweight": "bold",
    "legend.fontsize": 12,
    "legend.frameon": True,
    "legend.title_fontsize": 13,
    "lines.linewidth": 2.6,
    "lines.markersize": 6.5,
})
def style_axes(ax):
    ax.tick_params(axis="both", labelsize=14, width=2.0, length=6)
    for s in ax.spines.values():
        s.set_linewidth(2.0)

# ----------- HELPERS -----------
def safe_name(s: str) -> str:
    return re.sub(r"[^a-zA-Z0-9_-]+", "_", str(s)).strip("_")

def _normalize_name(s: str) -> str:
    return re.sub(r"\s+", " ", str(s).strip().lower())

def _resolve_sheet_name(xls: pd.ExcelFile, requested: str) -> str:
    """Find sheet by name, ignoring case and extra whitespace; supports partial match."""
    req = _normalize_name(requested)
    mapping = {_normalize_name(n): n for n in xls.sheet_names}
    if req in mapping:
        return mapping[req]
    # partial contains match
    for k, v in mapping.items():
        if req in k:
            return v
    raise ValueError(f"Sheet '{requested}' not found. Options: {xls.sheet_names}")

def _flatten_columns(df: pd.DataFrame) -> pd.DataFrame:
    """Flatten multirow headers, normalize whitespace, and deduplicate duplicate names."""
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = [
            " ".join([str(x) for x in tup if (str(x) != "nan" and str(x).strip() != "")]).strip()
            for tup in df.columns.to_list()
        ]
    df.columns = [re.sub(r"\s+", " ", str(c)).strip() for c in df.columns]
    seen = {}
    newcols = []
    for c in df.columns:
        if c in seen:
            seen[c] += 1
            newcols.append(f"{c}.{seen[c]}")
        else:
            seen[c] = 0
            newcols.append(c)
    df.columns = newcols
    return df

def _pick_col(df, patterns, required=True, exclude_regex=None):
    low = {c: str(c).strip().lower() for c in df.columns}
    for pat in patterns:
        rx = re.compile(pat)
        for c, lc in low.items():
            if rx.search(lc):
                if exclude_regex and re.search(exclude_regex, lc):
                    continue
                return c
    if required:
        raise KeyError(f"Could not find any of: {patterns}")
    return None

def find_year(df):  # prefer 'Year'
    return _pick_col(df, [r"\byear\b", r"^yr$", r"\bcalendar\s*year\b"])

def find_group(df):
    """Return the grouping column (Sex/Race/Age group/Region/Urbanization/State/Variable) or None for overall-only."""
    exclude = r"age\s*adjust|aamr|rate|ci|confidence|se|std|mean|median|total|overall"
    patterns = [
        r"\bvariable\b",
        r"\bage\s*group(s)?\b|age\s*cat(egory|egories)?\b|age\s*grp\b|age\-group",
        r"\bgender\b|\bsex\b",
        r"\brace\b|ethnic|hispanic",
        r"\bregion\b|census",
        r"\burban(ization)?\b|rural\b",
        r"\bstate\b|^us state$|^state name$",
        r"\boverall\b|^total$"
    ]
    for pat in patterns:
        try:
            c = _pick_col(df, [pat], required=False, exclude_regex=exclude)
            if c:
                return c
        except Exception:
            pass
    return None

def find_aamr(df):
    """
    Choose the most likely Age-Adjusted Rate column (numeric).
    - prefer exact 'Age Adjusted Rate' / 'AAMR' / 'Age standardized rate'
    - otherwise any column containing both 'age'+'adjust' and 'rate'
    - if multiple candidates, pick the one with the largest variance
    """
    low = {c: str(c).strip().lower() for c in df.columns}

    candidates = []
    exact_pats = [
        r"\bage[-\s]*adjust(ed)?\s*rate\b",
        r"\baamr\b",
        r"\bage[-\s]*standard(ized)?\s*rate\b",
    ]
    for p in exact_pats:
        try:
            c = _pick_col(df, [p], required=False)
            if c is not None:
                candidates.append(c)
        except Exception:
            pass

    if not candidates:
        # fallback: both 'age'+'adjust' and 'rate' in name, exclude crude
        for c, lc in low.items():
            if all(k in lc for k in ["age", "adjust"]) and "rate" in lc and "crude" not in lc:
                candidates.append(c)

    # keep only numeric
    candidates = [c for c in candidates if pd.api.types.is_numeric_dtype(df[c])]

    if not candidates:
        raise KeyError("No numeric Age-Adjusted Rate column found.")

    if len(candidates) == 1:
        return candidates[0]

    # pick the one with highest variance (more informative)
    variances = {c: pd.to_numeric(df[c], errors="coerce").var(skipna=True) for c in candidates}
    return max(variances, key=variances.get)

def load_observed_excel(path, sheet_name, overall_label="Overall"):
    """Return tidy dataframe: Year, Group, AAMR."""
    xls = pd.ExcelFile(path)
    real_sheet = _resolve_sheet_name(xls, sheet_name)

    df = pd.read_excel(xls, sheet_name=real_sheet)
    df = _flatten_columns(df)

    ycol = find_year(df)
    tcol = find_aamr(df)
    gcol = find_group(df)  # may be None

    df[ycol] = pd.to_numeric(df[ycol], errors="coerce")
    df[tcol] = pd.to_numeric(df[tcol], errors="coerce")

    if gcol and gcol == tcol:  # safety
        gcol = None

    if gcol is None:
        tidy = (df[[ycol, tcol]]
                .dropna(subset=[ycol, tcol])
                .groupby(ycol, as_index=False)[tcol].mean()
                .rename(columns={ycol: "Year", tcol: "AAMR"}))
        tidy["Group"] = overall_label
        tidy = tidy[["Year", "Group", "AAMR"]]
    else:
        df[gcol] = df[gcol].astype(str).str.strip()
        tidy = (df[[ycol, gcol, tcol]]
                .dropna(subset=[ycol, gcol, tcol])
                .groupby([ycol, gcol], as_index=False)[tcol].mean()
                .rename(columns={ycol: "Year", gcol: "Group", tcol: "AAMR"}))

    tidy["Year"] = tidy["Year"].astype(int)
    tidy = tidy.sort_values(["Group", "Year"]).reset_index(drop=True)

    print(f"\nSheet: {real_sheet}")
    print(f"Detected → Year: '{ycol}' | AAMR: '{tcol}' | Group: '{(gcol or overall_label)}'")
    print(tidy.head(6))
    return tidy

# ----------- ARIMA + FORECAST -----------
def _sanitize_series_for_arima(y: pd.Series):
    y = y.sort_index()
    full = pd.Index(range(int(y.index.min()), int(y.index.max()) + 1), name=y.index.name)
    y = y.reindex(full)
    if y.isna().any():
        y = y.interpolate(limit_direction="both")
    return y

def select_arima_order(y):
    best = None
    for d in [0, 1, 2]:
        for p in range(0, 4):
            for q in range(0, 4):
                if (p, d, q) == (0, 0, 0):
                    continue
                try:
                    trend = "n" if d > 0 else "c"
                    res = ARIMA(
                        y, order=(p, d, q), trend=trend,
                        enforce_stationarity=False, enforce_invertibility=False
                    ).fit(method_kwargs={"warn_convergence": False})
                    score = res.aic
                    if (best is None) or (score < best[0]):
                        best = (score, (p, d, q, trend), res)
                except Exception:
                    pass
    if best is None:
        res = ARIMA(y, order=(1, 1, 0), trend="n",
                    enforce_stationarity=False, enforce_invertibility=False
                   ).fit(method_kwargs={"warn_convergence": False})
        return (1, 1, 0, "n"), res
    return best[1], best[2]

def forecast_to(y, end_year, conf=0.95):
    last_year = int(y.index.max())
    steps = max(0, end_year - last_year)
    if steps == 0:
        raise ValueError("Observed series already extends to END_YEAR.")
    order, res = select_arima_order(y)
    fc = res.get_forecast(steps=steps)
    mean = fc.predicted_mean
    ci   = fc.conf_int(alpha=1 - conf)
    lo, hi = ci.iloc[:, 0], ci.iloc[:, 1]
    yrs = list(range(last_year + 1, end_year + 1))
    out = pd.DataFrame({
        "Year": yrs,
        "Point.Forecast": mean.values,
        "Lo.95": lo.values,
        "Hi.95": hi.values,
        "Order": [f"{order[0]},{order[1]},{order[2]} ({order[3]})"] * steps
    })
    return order, out

# ----------- FIT, SAVE, PLOT -----------
def fit_all_groups_and_save(observed_df, end_year, out_dir: Path, title_prefix="Forecast"):
    out_dir.mkdir(parents=True, exist_ok=True)
    groups = sorted(observed_df["Group"].unique())
    all_rows = []
    last_obs_year = int(observed_df["Year"].max())

    for g in groups:
        sub = observed_df[observed_df["Group"] == g].dropna(subset=["Year", "AAMR"]).sort_values("Year")
        y = pd.Series(sub["AAMR"].values, index=sub["Year"].astype(int), name="AAMR")
        y = _sanitize_series_for_arima(y)

        order, fc = forecast_to(y, end_year=end_year, conf=0.95)

        # Save per-group CSV (2 decimals)
        csv_path = out_dir / f"{safe_name(g)}_forecast_to_{end_year}.csv"
        fc_out = fc.copy()
        for c in ["Point.Forecast", "Lo.95", "Hi.95"]:
            fc_out[c] = pd.to_numeric(fc_out[c], errors="coerce").round(2)
        fc_out.insert(0, "Series", g)
        fc_out.to_csv(csv_path, index=False)
        print(f"[{g}] order={order}  -> CSV: {csv_path}")

        # Keep for consolidated
        tmp = fc_out.copy(); tmp["Group"] = g
        all_rows.append(tmp)

        # Per-group plot
        fig, ax = plt.subplots()
        ax.plot(y.index, y.values, marker="o", label=f"{g} (obs)")
        ln, = ax.plot(fc["Year"], fc["Point.Forecast"], linestyle="--", marker="o", label=f"{g} (fc)")
        ax.fill_between(fc["Year"], fc["Lo.95"], fc["Hi.95"], alpha=0.14, color=ln.get_color(), label="95% CI (fc)")
        ax.axvline(x=last_obs_year + 0.5, linestyle=":", linewidth=2.0)
        ax.set_title(f"{title_prefix}: {g} — observed (≤{last_obs_year}) & ARIMA forecast ({last_obs_year+1}–{end_year})")
        ax.set_xlabel("Year"); ax.set_ylabel("AAMR (per 100,000)")
        ax.yaxis.set_major_formatter(FormatStrFormatter("%.2f"))
        style_axes(ax)
        ax.legend(ncols=2)
        plt.tight_layout()
        png_path = out_dir / f"{safe_name(g)}_timeseries_to_{end_year}.png"
        plt.savefig(png_path, dpi=300, bbox_inches="tight"); plt.close()
        print("   Plot:", png_path)

    if all_rows:
        all_df = pd.concat(all_rows, ignore_index=True)
        all_df.to_csv(out_dir / f"ALL_GROUPS_forecasts_to_{end_year}.csv", index=False)
        print("Consolidated CSV:", out_dir / f"ALL_GROUPS_forecasts_to_{end_year}.csv")

def plot_combined(observed_df, out_dir: Path, end_year: int, title="Forecast", top_n=None):
    last_obs_year = int(observed_df["Year"].max())
    groups = sorted(observed_df["Group"].unique())

    # read saved forecasts for CI
    fc_map = {}
    for g in groups:
        p = out_dir / f"{safe_name(g)}_forecast_to_{end_year}.csv"
        if p.exists():
            fc_map[g] = pd.read_csv(p)

    # optionally limit to top N by last observed value
    if top_n is not None and len(groups) > top_n:
        latest = (observed_df.sort_values("Year")
                  .groupby("Group", as_index=False)
                  .apply(lambda d: d.iloc[-1][["Group", "AAMR"]])
                  .reset_index(drop=True))
        keep = (latest.sort_values("AAMR", ascending=False)
                      .head(top_n)["Group"].tolist())
        groups = [g for g in groups if g in keep]

    fig, ax = plt.subplots()
    ci_legend_added = False

    colors = plt.rcParams['axes.prop_cycle'].by_key().get('color', None)
    color_map = {}
    if colors:
        for i, g in enumerate(groups):
            color_map[g] = colors[i % len(colors)]

    for g in groups:
        sub = observed_df[observed_df["Group"] == g].copy().sort_values("Year")
        kwargs = {"marker": "o", "label": f"{g} (obs)"}
        if g in color_map: kwargs["color"] = color_map[g]
        ax.plot(sub["Year"], sub["AAMR"], **kwargs)

        if g in fc_map:
            fc = fc_map[g]
            kwargs_fc = {"linestyle": "--", "marker": "o", "label": f"{g} (fc)"}
            if g in color_map: kwargs_fc["color"] = color_map[g]
            ln, = ax.plot(fc["Year"], fc["Point.Forecast"], **kwargs_fc)
            # one legend entry for CI:
            if not ci_legend_added:
                ax.fill_between(fc["Year"], fc["Lo.95"], fc["Hi.95"], alpha=0.12,
                                color=ln.get_color(), label="95% CI (fc)")
                ci_legend_added = True
            else:
                ax.fill_between(fc["Year"], fc["Lo.95"], fc["Hi.95"], alpha=0.12,
                                color=ln.get_color())

    ax.axvline(x=last_obs_year + 0.5, linestyle=":", linewidth=2.0)
    ax.set_title(f"{title}: observed (≤{last_obs_year}) & ARIMA forecast ({last_obs_year+1}–{end_year})")
    ax.set_xlabel("Year"); ax.set_ylabel("AAMR (per 100,000)")
    ax.yaxis.set_major_formatter(FormatStrFormatter("%.2f"))
    style_axes(ax)
    ax.legend(ncols=2)
    ax.margins(x=0.02)
    plt.tight_layout()
    out = out_dir / f"COMBINED_timeseries_to_{end_year}.png"
    plt.savefig(out, dpi=300, bbox_inches="tight"); plt.close()
    print("Combined plot:", out)

# ----------- RUN ALL SHEETS -----------
for sheet_name, folder, pretty_title in RUN_SHEETS:
    out_dir = OUTPUT_BASE / folder
    out_dir.mkdir(parents=True, exist_ok=True)

    # 1) Load & tidy
    observed = load_observed_excel(FILE_PATH, sheet_name=sheet_name, overall_label="Overall")

    # 2) Fit & save per group (CSV + per-group plot)
    fit_all_groups_and_save(observed, END_YEAR, out_dir, title_prefix=pretty_title)

    # 3) Combined plot (optionally limit very large categories)
    topn = TOP_N_FOR_COMBINED.get(folder)
    plot_combined(observed, out_dir, END_YEAR, title=pretty_title, top_n=topn)



Sheet: overall final
Detected → Year: 'Year' | AAMR: 'Age Adjusted Rate' | Group: 'Overall'
   Year    Group      AAMR
0  1999  Overall  6.916667
1  2000  Overall  6.166667
2  2001  Overall  5.406667
3  2002  Overall  4.800000
4  2003  Overall  4.310000
5  2004  Overall  3.853333
[Overall] order=(1, 0, 0, 'c')  -> CSV: D:\arima project\Task3\Task 3\results\overall\Overall_forecast_to_2043.csv
   Plot: D:\arima project\Task3\Task 3\results\overall\Overall_timeseries_to_2043.png
Consolidated CSV: D:\arima project\Task3\Task 3\results\overall\ALL_GROUPS_forecasts_to_2043.csv
Combined plot: D:\arima project\Task3\Task 3\results\overall\COMBINED_timeseries_to_2043.png

Sheet: Race final 
Detected → Year: 'Year' | AAMR: 'Age Adjusted Rate' | Group: 'Hispanic Origin'
   Year                             Group  AAMR
0  1999  American Indian or Alaska Native  4.44
1  2000  American Indian or Alaska Native  4.81
2  2001  American Indian or Alaska Native  3.49
3  2002  American Indian or Alaska N

KeyError: "Could not find any of: ['\\\\byear\\\\b', '^yr$', '\\\\bcalendar\\\\s*year\\\\b']"

In [17]:
# ==== ARIMA FORECASTS & PLOTS TO 2043 (robust to 'Year Code' & Sex in 'overall final') ====
# !pip install pandas numpy matplotlib openpyxl statsmodels

import re, warnings
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter
from statsmodels.tsa.arima.model import ARIMA
warnings.filterwarnings("ignore")

# ----------- USER SETTINGS -----------
FILE_PATH   = r"D:\arima project\Task3\Task 3\cdc (IHD AND VFF) finals.xlsx"
OUTPUT_BASE = Path(r"D:\arima project\Task3\Task 3\results")
END_YEAR    = 2043

# Sheet list → (sheet_name_as_you_see_it, folder_name, pretty_title)
RUN_SHEETS = [
    ("overall final",        "overall",        "Overall"),
    ("Race final",           "race",           "Race / Ethnicity"),
    ("census final",         "region",         "Census Regions"),
    ("state final",          "state",          "States"),
    ("urbanization final",   "urbanization",   "Urbanization"),
    ("age group final",      "age",            "Age Groups"),
]

TOP_N_FOR_COMBINED = {
    "state": 12,   # show top 12 states in combined plot; set None to show all
}

# ----------- PLOT STYLE -----------
plt.rcParams.update({
    "figure.figsize": (13, 7.6),
    "figure.dpi": 160,
    "axes.grid": True,
    "grid.alpha": 0.25,
    "font.size": 14,
    "font.weight": "bold",
    "axes.labelsize": 18,
    "axes.labelweight": "bold",
    "axes.titlesize": 22,
    "axes.titleweight": "bold",
    "legend.fontsize": 12,
    "legend.frameon": True,
    "legend.title_fontsize": 13,
    "lines.linewidth": 2.6,
    "lines.markersize": 6.5,
})
def style_axes(ax):
    ax.tick_params(axis="both", labelsize=14, width=2.0, length=6)
    for s in ax.spines.values():
        s.set_linewidth(2.0)

# ----------- HELPERS -----------
def safe_name(s: str) -> str:
    return re.sub(r"[^a-zA-Z0-9_-]+", "_", str(s)).strip("_")

def _normalize_name(s: str) -> str:
    return re.sub(r"\s+", " ", str(s).strip().lower())

def _resolve_sheet_name(xls: pd.ExcelFile, requested: str) -> str:
    """Find sheet by name, ignoring case and extra whitespace; supports contains-match."""
    req = _normalize_name(requested)
    mapping = {_normalize_name(n): n for n in xls.sheet_names}
    if req in mapping:
        return mapping[req]
    for k, v in mapping.items():
        if req in k:
            return v
    raise ValueError(f"Sheet '{requested}' not found. Options: {xls.sheet_names}")

def _flatten_columns(df: pd.DataFrame) -> pd.DataFrame:
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = [
            " ".join([str(x) for x in tup if (str(x) != "nan" and str(x).strip() != "")]).strip()
            for tup in df.columns.to_list()
        ]
    df.columns = [re.sub(r"\s+", " ", str(c)).strip() for c in df.columns]
    # de-duplicate
    seen = {}
    newcols = []
    for c in df.columns:
        if c in seen:
            seen[c] += 1
            newcols.append(f"{c}.{seen[c]}")
        else:
            seen[c] = 0
            newcols.append(c)
    df.columns = newcols
    return df

def _pick_col(df, patterns, required=True, exclude_regex=None):
    low = {c: str(c).strip().lower() for c in df.columns}
    for pat in patterns:
        rx = re.compile(pat)
        for c, lc in low.items():
            if rx.search(lc):
                if exclude_regex and re.search(exclude_regex, lc):
                    continue
                return c
    if required:
        raise KeyError(f"Could not find any of: {patterns}")
    return None

def find_year(df):
    # common names first
    try: return _pick_col(df, [r"\byear\b", r"^yr$", r"\bcalendar\s*year\b"], required=False)
    except: pass
    # accept Year Code explicitly
    try: return _pick_col(df, [r"\byear\s*code\b"], required=False)
    except: pass
    # fallback: any column that contains 'year'
    for c in df.columns:
        if "year" in str(c).strip().lower():
            return c
    raise KeyError(r"Could not find any year column (e.g., 'Year', 'Year Code').")

def find_group(df):
    """Prefer a true grouping column (Sex, Race, Age group, Region, Urbanization, State, Variable)."""
    exclude = r"age\s*adjust|aamr|rate|ci|confidence|se|std|mean|median|total|overall"
    patterns = [
        r"\bvariable\b",
        r"\bage\s*group(s)?\b|age\s*cat(egory|egories)?\b|age\s*grp\b|age\-group",
        r"\bgender\b|\bsex\b",
        r"\brace\b|ethnic|hispanic",
        r"\bregion\b|census",
        r"\burban(ization)?\b|rural\b",
        r"\bstate\b|^us state$|^state name$",
    ]
    for pat in patterns:
        try:
            c = _pick_col(df, [pat], required=False, exclude_regex=exclude)
            if c:
                return c
        except Exception:
            pass
    return None

def find_aamr(df):
    # candidates by name
    exact_pats = [
        r"\bage[-\s]*adjust(ed)?\s*rate\b",
        r"\baamr\b",
        r"\bage[-\s]*standard(ized)?\s*rate\b",
    ]
    candidates = []
    for p in exact_pats:
        try:
            c = _pick_col(df, [p], required=False)
            if c is not None: candidates.append(c)
        except: pass
    if not candidates:
        # both 'age'+'adjust' and 'rate', but not crude
        for c in df.columns:
            lc = str(c).strip().lower()
            if all(k in lc for k in ["age","adjust"]) and "rate" in lc and "crude" not in lc:
                candidates.append(c)
    # keep numeric only
    candidates = [c for c in candidates if pd.api.types.is_numeric_dtype(df[c])]
    if not candidates:
        raise KeyError("No numeric Age-Adjusted Rate column found.")
    if len(candidates) == 1: return candidates[0]
    # choose with highest variance
    variances = {c: pd.to_numeric(df[c], errors="coerce").var(skipna=True) for c in candidates}
    return max(variances, key=variances.get)

def load_observed_excel(path, sheet_name, overall_label="Overall"):
    xls = pd.ExcelFile(path)
    real_sheet = _resolve_sheet_name(xls, sheet_name)
    df = pd.read_excel(xls, sheet_name=real_sheet)
    df = _flatten_columns(df)

    ycol = find_year(df)
    tcol = find_aamr(df)
    gcol = find_group(df)  # may be None

    # SPECIAL: if 'Sex' exists, use it (ensures 'overall final' splits into Male/Female)
    sex_cols = [c for c in df.columns if _normalize_name(c) == "sex"]
    if sex_cols:
        gcol = sex_cols[0]

    df[ycol] = pd.to_numeric(df[ycol], errors="coerce")
    df[tcol] = pd.to_numeric(df[tcol], errors="coerce")

    if gcol and gcol == tcol:
        gcol = None

    if gcol is None:
        tidy = (df[[ycol, tcol]]
                .dropna(subset=[ycol, tcol])
                .groupby(ycol, as_index=False)[tcol].mean()
                .rename(columns={ycol:"Year", tcol:"AAMR"}))
        tidy["Group"] = overall_label
        tidy = tidy[["Year","Group","AAMR"]]
    else:
        df[gcol] = df[gcol].astype(str).str.strip()
        tidy = (df[[ycol, gcol, tcol]]
                .dropna(subset=[ycol, gcol, tcol])
                .groupby([ycol, gcol], as_index=False)[tcol].mean()
                .rename(columns={ycol:"Year", gcol:"Group", tcol:"AAMR"}))

    tidy["Year"] = tidy["Year"].astype(int)
    tidy = tidy.sort_values(["Group","Year"]).reset_index(drop=True)

    print(f"\nSheet: {real_sheet}")
    print(f"Detected → Year: '{ycol}' | AAMR: '{tcol}' | Group: '{(gcol or overall_label)}'")
    print(tidy.head(6))
    return tidy

# ----------- ARIMA + FORECAST -----------
def _sanitize_series_for_arima(y: pd.Series):
    y = y.sort_index()
    full = pd.Index(range(int(y.index.min()), int(y.index.max()) + 1), name=y.index.name)
    y = y.reindex(full)
    if y.isna().any():
        y = y.interpolate(limit_direction="both")
    return y

def select_arima_order(y):
    best = None
    for d in [0,1,2]:
        for p in range(0,4):
            for q in range(0,4):
                if (p,d,q) == (0,0,0):
                    continue
                try:
                    trend = "n" if d>0 else "c"
                    res = ARIMA(y, order=(p,d,q), trend=trend,
                                enforce_stationarity=False, enforce_invertibility=False
                               ).fit(method_kwargs={"warn_convergence":False})
                    score = res.aic
                    if (best is None) or (score < best[0]):
                        best = (score, (p,d,q,trend), res)
                except Exception:
                    pass
    if best is None:
        res = ARIMA(y, order=(1,1,0), trend="n",
                    enforce_stationarity=False, enforce_invertibility=False
                   ).fit(method_kwargs={"warn_convergence":False})
        return (1,1,0,"n"), res
    return best[1], best[2]

def forecast_to(y, end_year, conf=0.95):
    last_year = int(y.index.max())
    steps = max(0, end_year - last_year)
    if steps == 0:
        raise ValueError("Observed series already extends to END_YEAR.")
    order, res = select_arima_order(y)
    fc = res.get_forecast(steps=steps)
    mean = fc.predicted_mean
    ci   = fc.conf_int(alpha=1-conf)
    lo, hi = ci.iloc[:,0], ci.iloc[:,1]
    yrs = list(range(last_year+1, end_year+1))
    out = pd.DataFrame({
        "Year": yrs,
        "Point.Forecast": mean.values,
        "Lo.95": lo.values,
        "Hi.95": hi.values,
        "Order": [f"{order[0]},{order[1]},{order[2]} ({order[3]})"]*steps
    })
    return order, out

# ----------- FIT, SAVE, PLOT -----------
def fit_all_groups_and_save(observed_df, end_year, out_dir: Path, title_prefix="Forecast"):
    out_dir.mkdir(parents=True, exist_ok=True)
    groups = sorted(observed_df["Group"].unique())
    all_rows = []
    last_obs_year = int(observed_df["Year"].max())

    for g in groups:
        sub = observed_df[observed_df["Group"]==g].dropna(subset=["Year","AAMR"]).sort_values("Year")
        y = pd.Series(sub["AAMR"].values, index=sub["Year"].astype(int), name="AAMR")
        y = _sanitize_series_for_arima(y)

        order, fc = forecast_to(y, end_year=end_year, conf=0.95)

        # per-group CSV (2 decimals)
        csv_path = out_dir / f"{safe_name(g)}_forecast_to_{end_year}.csv"
        fc_out = fc.copy()
        for c in ["Point.Forecast","Lo.95","Hi.95"]:
            fc_out[c] = pd.to_numeric(fc_out[c], errors="coerce").round(2)
        fc_out.insert(0, "Series", g)
        fc_out.to_csv(csv_path, index=False)
        print(f"[{g}] order={order}  -> CSV: {csv_path}")

        # store for consolidated
        tmp = fc_out.copy(); tmp["Group"] = g
        all_rows.append(tmp)

        # per-group plot
        fig, ax = plt.subplots()
        ax.plot(y.index, y.values, marker="o", label=f"{g} (obs)")
        ln, = ax.plot(fc["Year"], fc["Point.Forecast"], linestyle="--", marker="o", label=f"{g} (fc)")
        ax.fill_between(fc["Year"], fc["Lo.95"], fc["Hi.95"], alpha=0.14, color=ln.get_color(), label="95% CI (fc)")
        ax.axvline(x=last_obs_year+0.5, linestyle=":", linewidth=2.0)
        ax.set_title(f"{title_prefix}: {g} — observed (≤{last_obs_year}) & ARIMA forecast ({last_obs_year+1}–{end_year})")
        ax.set_xlabel("Year"); ax.set_ylabel("AAMR (per 100,000)")
        ax.yaxis.set_major_formatter(FormatStrFormatter("%.2f"))
        style_axes(ax)
        ax.legend(ncols=2)
        plt.tight_layout()
        png_path = out_dir / f"{safe_name(g)}_timeseries_to_{end_year}.png"
        plt.savefig(png_path, dpi=300, bbox_inches="tight"); plt.close()
        print("   Plot:", png_path)

    if all_rows:
        all_df = pd.concat(all_rows, ignore_index=True)
        all_df.to_csv(out_dir / f"ALL_GROUPS_forecasts_to_{end_year}.csv", index=False)
        print("Consolidated CSV:", out_dir / f"ALL_GROUPS_forecasts_to_{end_year}.csv")

def plot_combined(observed_df, out_dir:Path, end_year:int, title="Forecast", top_n=None):
    last_obs_year = int(observed_df["Year"].max())
    groups = sorted(observed_df["Group"].unique())

    # read saved forecasts
    fc_map = {}
    for g in groups:
        p = out_dir / f"{safe_name(g)}_forecast_to_{end_year}.csv"
        if p.exists():
            fc_map[g] = pd.read_csv(p)

    # limit to top N by latest observed value (if requested)
    if top_n is not None and len(groups) > top_n:
        latest = (observed_df.sort_values("Year")
                  .groupby("Group", as_index=False)
                  .apply(lambda d: d.iloc[-1][["Group","AAMR"]])
                  .reset_index(drop=True))
        keep = (latest.sort_values("AAMR", ascending=False)
                      .head(top_n)["Group"].tolist())
        groups = [g for g in groups if g in keep]

    fig, ax = plt.subplots()
    ci_legend_added = False

    colors = plt.rcParams['axes.prop_cycle'].by_key().get('color', None)
    color_map = {}
    if colors:
        for i, g in enumerate(groups):
            color_map[g] = colors[i % len(colors)]

    for g in groups:
        sub = observed_df[observed_df["Group"]==g].copy().sort_values("Year")
        kw_obs = {"marker": "o", "label": f"{g} (obs)"}
        if g in color_map: kw_obs["color"] = color_map[g]
        ax.plot(sub["Year"], sub["AAMR"], **kw_obs)

        if g in fc_map:
            fc = fc_map[g]
            kw_fc = {"linestyle": "--", "marker": "o", "label": f"{g} (fc)"}
            if g in color_map: kw_fc["color"] = color_map[g]
            ln, = ax.plot(fc["Year"], fc["Point.Forecast"], **kw_fc)
            if not ci_legend_added:
                ax.fill_between(fc["Year"], fc["Lo.95"], fc["Hi.95"], alpha=0.12,
                                color=ln.get_color(), label="95% CI (fc)")
                ci_legend_added = True
            else:
                ax.fill_between(fc["Year"], fc["Lo.95"], fc["Hi.95"], alpha=0.12,
                                color=ln.get_color())

    ax.axvline(x=last_obs_year+0.5, linestyle=":", linewidth=2.0)
    ax.set_title(f"{title}: observed (≤{last_obs_year}) & ARIMA forecast ({last_obs_year+1}–{end_year})")
    ax.set_xlabel("Year"); ax.set_ylabel("AAMR (per 100,000)")
    ax.yaxis.set_major_formatter(FormatStrFormatter("%.2f"))
    style_axes(ax)
    ax.legend(ncols=2)
    ax.margins(x=0.02)
    plt.tight_layout()
    out = out_dir / f"COMBINED_timeseries_to_{end_year}.png"
    plt.savefig(out, dpi=300, bbox_inches="tight"); plt.close()
    print("Combined plot:", out)

# ----------- RUN ALL SHEETS -----------
for sheet_name, folder, pretty_title in RUN_SHEETS:
    out_dir = OUTPUT_BASE / folder
    out_dir.mkdir(parents=True, exist_ok=True)

    observed = load_observed_excel(FILE_PATH, sheet_name=sheet_name, overall_label="Overall")
    fit_all_groups_and_save(observed, END_YEAR, out_dir, title_prefix=pretty_title)
    topn = TOP_N_FOR_COMBINED.get(folder)
    plot_combined(observed, out_dir, END_YEAR, title=pretty_title, top_n=topn)



Sheet: overall final
Detected → Year: 'Year' | AAMR: 'Age Adjusted Rate' | Group: 'Sex'
   Year   Group  AAMR
0  1999  Female  4.01
1  2000  Female  3.46
2  2001  Female  3.03
3  2002  Female  2.68
4  2003  Female  2.41
5  2004  Female  2.09
[Female] order=(1, 0, 1, 'c')  -> CSV: D:\arima project\Task3\Task 3\results\overall\Female_forecast_to_2043.csv
   Plot: D:\arima project\Task3\Task 3\results\overall\Female_timeseries_to_2043.png
[Male] order=(2, 0, 0, 'c')  -> CSV: D:\arima project\Task3\Task 3\results\overall\Male_forecast_to_2043.csv
   Plot: D:\arima project\Task3\Task 3\results\overall\Male_timeseries_to_2043.png
[overall] order=(1, 0, 0, 'c')  -> CSV: D:\arima project\Task3\Task 3\results\overall\overall_forecast_to_2043.csv
   Plot: D:\arima project\Task3\Task 3\results\overall\overall_timeseries_to_2043.png
Consolidated CSV: D:\arima project\Task3\Task 3\results\overall\ALL_GROUPS_forecasts_to_2043.csv
Combined plot: D:\arima project\Task3\Task 3\results\overall\COMBINED

KeyError: None

In [19]:
# ==== ARIMA FORECASTS & PLOTS → 2043 (robust + bold + 95% CI) ====
# If first time:
# !pip install pandas numpy matplotlib openpyxl statsmodels

import re, warnings
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter
from statsmodels.tsa.arima.model import ARIMA
warnings.filterwarnings("ignore")

# ---------------- USER SETTINGS ----------------
FILE_PATH   = r"D:\arima project\Task3\Task 3\cdc (IHD AND VFF) finals.xlsx"
OUTPUT_BASE = Path(r"D:\arima project\Task3\Task 3\results")
END_YEAR    = 2043

# Done: overall / race / region. Remaining commonly are: state, urbanization, age.
# You can run all; it will overwrite safely.
RUN_SHEETS = [

    ("urbanization final",   "urbanization",   "Urbanization"),
    ("age group final",      "age",            "Age Groups"),
]

# Limit combined chart to top-N by latest observed AAMR (e.g., many states).
TOP_N_FOR_COMBINED = {
    "state": 12,   # set None to show all
}

# ---------------- PLOT STYLE ----------------
plt.rcParams.update({
    "figure.figsize": (13, 7.6),
    "figure.dpi": 160,
    "axes.grid": True,
    "grid.alpha": 0.25,
    "font.size": 14,
    "font.weight": "bold",
    "axes.labelsize": 18,
    "axes.labelweight": "bold",
    "axes.titlesize": 22,
    "axes.titleweight": "bold",
    "legend.fontsize": 12,
    "legend.frameon": True,
    "legend.title_fontsize": 13,
    "lines.linewidth": 2.6,
    "lines.markersize": 6.5,
})
def style_axes(ax):
    ax.tick_params(axis="both", labelsize=14, width=2.0, length=6)
    for s in ax.spines.values():
        s.set_linewidth(2.0)

# ---------------- HELPERS ----------------
def safe_name(s: str) -> str:
    return re.sub(r"[^a-zA-Z0-9_-]+", "_", str(s)).strip("_")

def _normalize_name(s: str) -> str:
    return re.sub(r"\s+", " ", str(s).strip().lower())

def _resolve_sheet_name(xls: pd.ExcelFile, requested: str) -> str:
    """Find sheet ignoring case/extra spaces; supports contains-match."""
    req = _normalize_name(requested)
    mapping = {_normalize_name(n): n for n in xls.sheet_names}
    if req in mapping:
        return mapping[req]
    for k, v in mapping.items():
        if req in k:
            return v
    raise ValueError(f"Sheet '{requested}' not found. Options: {xls.sheet_names}")

def _flatten_columns(df: pd.DataFrame) -> pd.DataFrame:
    """Flatten multirow headers, normalize whitespace, deduplicate names."""
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = [
            " ".join([str(x) for x in tup if (str(x) != "nan" and str(x).strip() != "")]).strip()
            for tup in df.columns.to_list()
        ]
    df.columns = [re.sub(r"\s+", " ", str(c)).strip() for c in df.columns]
    seen, newcols = {}, []
    for c in df.columns:
        if c in seen:
            seen[c] += 1; newcols.append(f"{c}.{seen[c]}")
        else:
            seen[c] = 0;  newcols.append(c)
    df.columns = newcols
    return df

def _pick_col(df, patterns, required=True, exclude_regex=None):
    low = {c: str(c).strip().lower() for c in df.columns}
    for pat in patterns:
        rx = re.compile(pat)
        for c, lc in low.items():
            if rx.search(lc):
                if exclude_regex and re.search(exclude_regex, lc):
                    continue
                return c
    if required:
        raise KeyError(f"Could not find any of: {patterns}")
    return None

def find_year(df):
    """
    Robust: 'Year', 'Year Code', 'Calendar Year', or any header containing 'year'.
    Avoid returning None; raise with clear message if not found.
    """
    # exact-ish first
    y = _pick_col(df, [r"\byear\b", r"^yr$", r"\bcalendar\s*year\b"], required=False)
    if y: return y
    y = _pick_col(df, [r"\byear\s*code\b", r"\bdata\s*year\b", r"\byear\s*of\s*death\b"], required=False)
    if y: return y
    # fallback: any column containing 'year'
    for c in df.columns:
        if "year" in str(c).strip().lower():
            return c
    # last resort: helpful error
    raise KeyError(
        "No year column found. Expected headers like 'Year', 'Year Code', 'Calendar Year'. "
        f"Available columns: {list(df.columns)}"
    )

def find_group(df):
    """Return grouping col (Sex, Race, Age group, Region, Urbanization, State, Variable) or None."""
    exclude = r"age\s*adjust|aamr|rate|ci|confidence|se|std|mean|median|total|overall"
    patterns = [
        r"\bvariable\b",
        r"\bage\s*group(s)?\b|age\s*cat(egory|egories)?\b|age\s*grp\b|age\-group",
        r"\bgender\b|\bsex\b",
        r"\brace\b|ethnic|hispanic",
        r"\bregion\b|census",
        r"\burban(ization)?\b|metro|non[-\s]?metro|rural\b",
        r"\bstate\b|^us state$|^state name$",
    ]
    for pat in patterns:
        c = _pick_col(df, [pat], required=False, exclude_regex=exclude)
        if c:
            return c
    return None

def find_aamr(df):
    """
    Choose Age-Adjusted Rate column:
    - exact names first (Age Adjusted Rate / AAMR / Age standardized rate / Age-adjusted death rate)
    - else any column name containing 'age' & 'adjust' & 'rate' (not 'crude')
    - among candidates, pick numeric with largest variance
    """
    exact_pats = [
        r"\bage[-\s]*adjust(ed)?\s*rate\b",
        r"\bage[-\s]*adjust(ed)?\s*death\s*rate\b",
        r"\baamr\b",
        r"\bage[-\s]*standard(ized)?\s*rate\b",
    ]
    candidates = []
    for p in exact_pats:
        c = _pick_col(df, [p], required=False)
        if c: candidates.append(c)

    if not candidates:
        for c in df.columns:
            lc = str(c).strip().lower()
            if all(k in lc for k in ["age","adjust"]) and "rate" in lc and "crude" not in lc:
                candidates.append(c)

    # numeric only
    num_cands = [c for c in candidates if pd.api.types.is_numeric_dtype(df[c])]
    if not num_cands:
        raise KeyError(
            "No numeric Age-Adjusted Rate column found. "
            f"Tried candidates: {candidates} | Columns: {list(df.columns)}"
        )

    if len(num_cands) == 1:
        return num_cands[0]

    variances = {c: pd.to_numeric(df[c], errors="coerce").var(skipna=True) for c in num_cands}
    return max(variances, key=variances.get)

def _rate_axis_label_from_header(colname: str) -> str:
    s = str(colname).lower()
    if "1,000,000" in colname or "per 1,000,000" in s or "per million" in s:
        return "AAMR (per 1,000,000)"
    return "AAMR (per 100,000)"

def load_observed_excel(path, sheet_name, overall_label="Overall"):
    """Return tidy dataframe: Year, Group, AAMR (mean if duplicates)."""
    xls = pd.ExcelFile(path)
    real_sheet = _resolve_sheet_name(xls, sheet_name)
    df = pd.read_excel(xls, sheet_name=real_sheet)
    df = _flatten_columns(df)

    ycol = find_year(df)
    tcol = find_aamr(df)
    gcol = find_group(df)  # may be None

    # SPECIAL: if 'Sex' exists anywhere, force it as group (ensures 'overall final' splits Male/Female).
    sex_cols = [c for c in df.columns if _normalize_name(c) == "sex"]
    if sex_cols:
        gcol = sex_cols[0]

    df[ycol] = pd.to_numeric(df[ycol], errors="coerce")
    df[tcol] = pd.to_numeric(df[tcol], errors="coerce")

    if gcol and gcol == tcol:
        gcol = None

    if gcol is None:
        tidy = (df[[ycol, tcol]]
                .dropna(subset=[ycol, tcol])
                .groupby(ycol, as_index=False)[tcol].mean()
                .rename(columns={ycol:"Year", tcol:"AAMR"}))
        tidy["Group"] = overall_label
        tidy = tidy[["Year","Group","AAMR"]]
    else:
        df[gcol] = df[gcol].astype(str).str.strip()
        tidy = (df[[ycol, gcol, tcol]]
                .dropna(subset=[ycol, gcol, tcol])
                .groupby([ycol, gcol], as_index=False)[tcol].mean()
                .rename(columns={ycol:"Year", gcol:"Group", tcol:"AAMR"}))

    tidy["Year"] = tidy["Year"].astype(int)
    tidy = tidy.sort_values(["Group","Year"]).reset_index(drop=True)

    # keep label to format y-axis later
    tidy._aamr_label = _rate_axis_label_from_header(tcol)

    print(f"\nSheet: {real_sheet}")
    print(f"Detected → Year: '{ycol}' | AAMR: '{tcol}' | Group: '{(gcol or overall_label)}'")
    print(tidy.head(6))
    return tidy

# ---------------- ARIMA + FORECAST ----------------
def _sanitize_series_for_arima(y: pd.Series):
    y = y.sort_index()
    full = pd.Index(range(int(y.index.min()), int(y.index.max()) + 1), name=y.index.name)
    y = y.reindex(full)
    if y.isna().any():
        y = y.interpolate(limit_direction="both")
    return y

def select_arima_order(y):
    best = None
    for d in [0,1,2]:
        for p in range(0,4):
            for q in range(0,4):
                if (p,d,q) == (0,0,0): 
                    continue
                try:
                    trend = "n" if d>0 else "c"
                    res = ARIMA(
                        y, order=(p,d,q), trend=trend,
                        enforce_stationarity=False, enforce_invertibility=False
                    ).fit(method_kwargs={"warn_convergence":False})
                    score = res.aic
                    if (best is None) or (score < best[0]):
                        best = (score, (p,d,q,trend), res)
                except Exception:
                    pass
    if best is None:
        res = ARIMA(y, order=(1,1,0), trend="n",
                    enforce_stationarity=False, enforce_invertibility=False
                   ).fit(method_kwargs={"warn_convergence":False})
        return (1,1,0,"n"), res
    return best[1], best[2]

def forecast_to(y, end_year, conf=0.95):
    last_year = int(y.index.max())
    steps = max(0, end_year - last_year)
    if steps == 0:
        raise ValueError("Observed series already extends to END_YEAR.")
    order, res = select_arima_order(y)
    fc = res.get_forecast(steps=steps)
    mean = fc.predicted_mean
    ci   = fc.conf_int(alpha=1-conf)
    lo, hi = ci.iloc[:,0], ci.iloc[:,1]
    yrs = list(range(last_year+1, end_year+1))
    out = pd.DataFrame({
        "Year": yrs,
        "Point.Forecast": mean.values,
        "Lo.95": lo.values,
        "Hi.95": hi.values,
        "Order": [f"{order[0]},{order[1]},{order[2]} ({order[3]})"]*steps
    })
    return order, out

# ---------------- FIT, SAVE, PLOT ----------------
def fit_all_groups_and_save(observed_df, end_year, out_dir: Path, title_prefix="Forecast"):
    out_dir.mkdir(parents=True, exist_ok=True)
    groups = sorted(observed_df["Group"].unique())
    all_rows = []
    last_obs_year = int(observed_df["Year"].max())
    y_label = getattr(observed_df, "_aamr_label", "AAMR")

    for g in groups:
        sub = observed_df[observed_df["Group"]==g].dropna(subset=["Year","AAMR"]).sort_values("Year")
        y = pd.Series(sub["AAMR"].values, index=sub["Year"].astype(int), name="AAMR")
        y = _sanitize_series_for_arima(y)

        order, fc = forecast_to(y, end_year=end_year, conf=0.95)

        # per-group CSV (2 decimals)
        csv_path = out_dir / f"{safe_name(g)}_forecast_to_{end_year}.csv"
        fc_out = fc.copy()
        for c in ["Point.Forecast","Lo.95","Hi.95"]:
            fc_out[c] = pd.to_numeric(fc_out[c], errors="coerce").round(2)
        fc_out.insert(0, "Series", g)
        fc_out.to_csv(csv_path, index=False)
        print(f"[{g}] order={order}  -> CSV: {csv_path}")

        # store for consolidated
        tmp = fc_out.copy(); tmp["Group"] = g
        all_rows.append(tmp)

        # per-group plot
        fig, ax = plt.subplots()
        ax.plot(y.index, y.values, marker="o", label=f"{g} (obs)")
        ln, = ax.plot(fc["Year"], fc["Point.Forecast"], linestyle="--", marker="o", label=f"{g} (fc)")
        ax.fill_between(fc["Year"], fc["Lo.95"], fc["Hi.95"], alpha=0.14, color=ln.get_color(), label="95% CI (fc)")
        ax.axvline(x=last_obs_year+0.5, linestyle=":", linewidth=2.0)
        ax.set_title(f"{title_prefix}: {g} — observed (≤{last_obs_year}) & ARIMA forecast ({last_obs_year+1}–{end_year})")
        ax.set_xlabel("Year"); ax.set_ylabel(y_label)
        ax.yaxis.set_major_formatter(FormatStrFormatter("%.2f"))
        style_axes(ax)
        ax.legend(ncols=2)
        plt.tight_layout()
        png_path = out_dir / f"{safe_name(g)}_timeseries_to_{end_year}.png"
        plt.savefig(png_path, dpi=300, bbox_inches="tight"); plt.close()
        print("   Plot:", png_path)

    if all_rows:
        all_df = pd.concat(all_rows, ignore_index=True)
        all_df.to_csv(out_dir / f"ALL_GROUPS_forecasts_to_{end_year}.csv", index=False)
        print("Consolidated CSV:", out_dir / f"ALL_GROUPS_forecasts_to_{end_year}.csv")

def plot_combined(observed_df, out_dir:Path, end_year:int, title="Forecast", top_n=None):
    last_obs_year = int(observed_df["Year"].max())
    groups = sorted(observed_df["Group"].unique())
    y_label = getattr(observed_df, "_aamr_label", "AAMR")

    # read saved forecasts
    fc_map = {}
    for g in groups:
        p = out_dir / f"{safe_name(g)}_forecast_to_{end_year}.csv"
        if p.exists():
            fc_map[g] = pd.read_csv(p)

    # limit to top N (if requested)
    if top_n is not None and len(groups) > top_n:
        latest = (observed_df.sort_values("Year")
                  .groupby("Group", as_index=False)
                  .apply(lambda d: d.iloc[-1][["Group","AAMR"]])
                  .reset_index(drop=True))
        keep = (latest.sort_values("AAMR", ascending=False)
                      .head(top_n)["Group"].tolist())
        groups = [g for g in groups if g in keep]

    fig, ax = plt.subplots()
    ci_legend_added = False

    colors = plt.rcParams['axes.prop_cycle'].by_key().get('color', None)
    color_map = {}
    if colors:
        for i, g in enumerate(groups):
            color_map[g] = colors[i % len(colors)]

    for g in groups:
        sub = observed_df[observed_df["Group"]==g].copy().sort_values("Year")
        kw_obs = {"marker": "o", "label": f"{g} (obs)"}
        if g in color_map: kw_obs["color"] = color_map[g]
        ax.plot(sub["Year"], sub["AAMR"], **kw_obs)

        if g in fc_map:
            fc = fc_map[g]
            kw_fc = {"linestyle": "--", "marker": "o", "label": f"{g} (fc)"}
            if g in color_map: kw_fc["color"] = color_map[g]
            ln, = ax.plot(fc["Year"], fc["Point.Forecast"], **kw_fc)
            if not ci_legend_added:
                ax.fill_between(fc["Year"], fc["Lo.95"], fc["Hi.95"], alpha=0.12,
                                color=ln.get_color(), label="95% CI (fc)")
                ci_legend_added = True
            else:
                ax.fill_between(fc["Year"], fc["Lo.95"], fc["Hi.95"], alpha=0.12,
                                color=ln.get_color())

    ax.axvline(x=last_obs_year+0.5, linestyle=":", linewidth=2.0)
    ax.set_title(f"{title}: observed (≤{last_obs_year}) & ARIMA forecast ({last_obs_year+1}–{end_year})")
    ax.set_xlabel("Year"); ax.set_ylabel(y_label)
    ax.yaxis.set_major_formatter(FormatStrFormatter("%.2f"))
    style_axes(ax)
    ax.legend(ncols=2)
    ax.margins(x=0.02)
    plt.tight_layout()
    out = out_dir / f"COMBINED_timeseries_to_{end_year}.png"
    plt.savefig(out, dpi=300, bbox_inches="tight"); plt.close()
    print("Combined plot:", out)

# ---------------- RUN ALL SHEETS ----------------
for sheet_name, folder, pretty_title in RUN_SHEETS:
    out_dir = OUTPUT_BASE / folder
    out_dir.mkdir(parents=True, exist_ok=True)

    observed = load_observed_excel(FILE_PATH, sheet_name=sheet_name, overall_label="Overall")
    fit_all_groups_and_save(observed, END_YEAR, out_dir, title_prefix=pretty_title)

    topn = TOP_N_FOR_COMBINED.get(folder)
    plot_combined(observed, out_dir, END_YEAR, title=pretty_title, top_n=topn)



Sheet: urbanization final
Detected → Year: 'Year' | AAMR: 'Age Adjusted Rate' | Group: 'urbanization'
   Year  Group  AAMR
0  1999  Rural  7.20
1  2000  Rural  6.44
2  2001  Rural  5.62
3  2002  Rural  5.09
4  2003  Rural  4.49
5  2004  Rural  3.88
[Rural] order=(1, 0, 2, 'c')  -> CSV: D:\arima project\Task3\Task 3\results\urbanization\Rural_forecast_to_2043.csv
   Plot: D:\arima project\Task3\Task 3\results\urbanization\Rural_timeseries_to_2043.png
[Urban] order=(2, 0, 1, 'c')  -> CSV: D:\arima project\Task3\Task 3\results\urbanization\Urban_forecast_to_2043.csv
   Plot: D:\arima project\Task3\Task 3\results\urbanization\Urban_timeseries_to_2043.png
Consolidated CSV: D:\arima project\Task3\Task 3\results\urbanization\ALL_GROUPS_forecasts_to_2043.csv
Combined plot: D:\arima project\Task3\Task 3\results\urbanization\COMBINED_timeseries_to_2043.png

Sheet: age group final
Detected → Year: 'Year' | AAMR: 'Age Adjusted Rate' | Group: 'Age group'
   Year         Group  AAMR
0  1999  middle